# Midterm project
## Q1
$forwardSubeps.m$

```
function [ x ] = forwardSubeps( L,b )

%% Checking the input
if ~ismatrix(L)
    return;
end

if ~isvector(b)
    return;
else
    if isrow(b)
        b = b';
    end
end

szL = size(L);
n = szL(1);
if szL(1)~=szL(2)
    fprintf('Not a square matrix!\n');
    return;
end

U = triu(L,1);
if max(abs(U(:))) >= eps
    fprintf('Not a lower triangular matrix!\n');
    return;
end

d = diag(L);
if all(d(:)) == 0
    fprintf('Not a singular matrix!\n');
    return;
end

szb = size(b);
m = szb(1);
if m ~= n
    fprintf('Scale not match!\n');
    return;
end

%% Forward substitution
x = zeros(n,1);
for i = 1:n
    x(i) = b(i)/L(i,i);
    for j = (i+1):n
        b(j) = b(j) - L(j,i)*x(i);
    end
end

end
```

$backwardSubeps.m$

```
function [ x ] = backwardSubeps( U,b )
%UNTITLED Summary of this function goes here
%   Detailed explanation goes here

%% Checking the input
if ~ismatrix(U)
    return;
end

if ~isvector(b)
    return;
else
    if isrow(b)
        b = b';
    end
end

szU = size(U);
n = szU(1);
if szU(1)~=szU(2)
    fprintf('Not a square matrix!\n');
    return;
end

L = tril(U,-1);

if max(abs(L(:))) >= eps
    fprintf('Not a upper triangular matrix!\n');
    return;
end

d = diag(U);
if all(d(:)) == 0
    fprintf('Not a singular matrix!\n');
    return;
end


szb = size(b);
m = szb(1);
if m ~= n
    fprintf('Scale not match!\n');
    return;
end

%% Backward substitution
x = zeros(n,1);
for i = fliplr(1:n)
    x(i) = b(i)/U(i,i);
    for j = 1:(i-1)
        b(j) = b(j) - U(j,i)*x(i);
    end
end

end
```

## Q2
$computeLU.m$

```
function [ L,U ] = computeLU( A )
%UNTITLED3 Summary of this function goes here
%   Detailed explanation goes here
szA = size(A);
n = szA(1);
if szA(1)~=szA(2)
    fprintf('Not a square matrix!\n');
    return;
end

if rank(A) ~= n
    fprintf('Not a singular matrix!\n');
    return;
end   

L = diag(ones(1,n));
U = A;

for i = 1:(n-1)
    tempL = diag(ones(1,n));
    tempU = diag(ones(1,n));
    for j = (i+1):n
        tempL(j,i) =  U(j,i)/U(i,i);
        tempU(j,i) = -U(j,i)/U(i,i);
    end
    L = L*tempL;
    U = tempU*U;
end

L = tril(L);
U = triu(U);
```

## Q3
$computeLUpivot.m$

```
function [ L,U,p ] = computeLUpivot( A )
%UNTITLED3 Summary of this function goes here
%   Detailed explanation goes here
szA = size(A);
n = szA(1);
if szA(1)~=szA(2)
    fprintf('Not a square matrix!\n');
    return;
end

if rank(A) ~= n
    fprintf('Not a singular matrix!\n');
    return;
end   

L = eye(n);
U = A;
p = 1:n;
P = eye(n);
```
```
for i = 1:(n-1)    
    % Pivoting
    [maxv,maxp] = max(abs(U(i:n,i)));
    j = maxp + i - 1;
    if j ~= i
        temprow = U(i,:);
        U(i,:) = U(j,:);
        U(j,:) = temprow;
        temprow = L(:,i);
        L(:,i) = L(:,j);
        L(:,j) = temprow;
        temprow = P(i,:);
        P(i,:) = P(j,:);
        P(j,:) = temprow;
        tempv = p(i);
        p(i) = p(j);
        p(j) = tempv;
    end

    
    tempL = eye(n);
    tempU = eye(n);
    for j = (i+1):n
        tempL(j,i) =  U(j,i)/U(i,i);
        tempU(j,i) = -U(j,i)/U(i,i);
    end
    L = L*tempL;
    U = tempU*U;
    
end

L = P*L;
L = tril(L);
U = triu(U);

clear maxp
clear maxv
clear temprow
clear tempv
```

## Q4
$findSolution.m$

```
function [ x ] = findSolution( A,b )
%UNTITLED3 Summary of this function goes here
%   Detailed explanation goes here

[ L,U ] = computeLU( A );
[ y ] = forwardSubeps( L,b );
[ x ] = backwardSubeps( U,y );

end
```

$findSolutionPivot.m$

```
function [ x ] = findSolutionPivot( A,b )
%UNTITLED4 Summary of this function goes here
%   Detailed explanation goes here

[ L,U,p ] = computeLUpivot( A );

b2 = b;
for i = 1:size(b2,1)
    b(i) = b2(p(i));
end

[ y ] = forwardSubeps( L,b );
[ x ] = backwardSubeps( U,y );

end
```

## Q5
$midFive.m$

```
clear
clc

N = linspace(10,200,20);
errBackSlash = ones(20,1);
errFindSolution = ones(20,1);
errFindSolutionPivot = ones(20,1);

for i = 1:20
    n = N(i);
    ansTrue = ones(n,1);
    
    tempErrBackSlash = ones(5:1);
    tempErrFindSolution = ones(5:1);
    tempErrFindSolutionPivot = ones(5:1);
    
    for j = 1:5
        A = rand(n);
        b = A * ansTrue;
        
        x1 = A\b;
        x2 = findSolution(A,b);
        x3 = findSolutionPivot(A,b);
        
        tempErrBackSlash(j) = norm(x1-ansTrue);
        tempErrFindSolution(j) = norm(x2-ansTrue);
        tempErrFindSolutionPivot(j) = norm(x3-ansTrue);
    end
    
    errBackSlash(i) = mean(tempErrBackSlash);
    errFindSolution(i) = mean(tempErrFindSolution);
    errFindSolutionPivot(i) = mean(tempErrFindSolutionPivot);
end
```
```
rangeX = 1:20;
figure
s = semilogy(rangeX,errBackSlash,rangeX,errFindSolutionPivot,rangeX,errFindSolution);
% figure
% s1 = semilogy(rangeX,errBackSlash);
% hold on
% s2 = semilogy(rangeX,errFindSolutionPivot);
% s3 = semilogy(rangeX,errFindSolution);

s(1).LineWidth = 1;
s(2).LineWidth = 1.5;
s(3).LineWidth = 2;

axis([0 21 -inf inf])

legend([s(1) s(2) s(3)],'\fontname{Courier New} \bf Error Using Backslash',...
    '\fontname{Courier New} \bf Error Using Gassian Elimination with pivoting',...
    '\fontname{Courier New} \bf Error Using Gassian Elimination without pivoting',...
    'Location','northwest')

Y = ylabel('\fontname{Courier New} \bf Err');
X = xlabel('\fontname{Courier New} \bf size');

set(Y, 'Units', 'Normalized', 'Position', [-0.07, 0.5, 0],'Rotation',0);
set(X, 'Units', 'Normalized', 'Position', [0.5, -0.07, 0]);
```

**Result**
![](./Mid/midFive.png)